In [1]:
import os

In [2]:
%pwd

'd:\\project\\chicken-fecal-classification_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\project\\chicken-fecal-classification_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorbord_root_log_pair: Path  # Correct attribute name
    checkpoin_model_filepath: Path


In [6]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml,create_directories

In [7]:
# class ConfigurationManager:
#     def __init__(self,
#                  config_filepath = CONFIG_FILE_PATH,
#                  params_filepath = PARAMS_FILE_PATH):
#         self.config = read_yaml(config_filepath)
#         self.params = read_yaml(params_filepath)
#         create_directories([self.config.artifacts_root])


#     def get_prepare_base_model_config(self) -> PrepareCallbacksConfig:
#         config = self.config.prepare_callbacks
#         model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
#         create_directories([
#             Path(model_ckpt_dir),
#             Path(config.tensorboard_root_log_dir)
#         ])
#         Prepare_callbacks_Config = PrepareCallbacksConfig(
#             root_dir=Path(config.root.dir),
#             tensorbord_root_log_dir = Path(config.tensorboard_root_log_dir),
#             checkpoin_model_filepath=Path(config.checkpoint_model_filepath)
#         )
#         return Prepare_callbacks_Config

In [8]:
from pathlib import Path

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Ensure artifacts_root is created as a directory
        artifacts_root = Path(self.config["artifacts_root"])
        create_directories([artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareCallbacksConfig:
        # Access the nested prepare_base_model section
        config = self.config["data_ingestion"]["prepare_base_model"]

        # Convert paths to Path objects
        model_ckpt_dir = Path(config["updated_base_model_path"]).parent
        tensorboard_root_log_dir = Path(config["root_dir"]) / "logs"
        
        create_directories([
            model_ckpt_dir,
            tensorboard_root_log_dir,
        ])

        # Create and return PrepareCallbacksConfig with Path objects
        return PrepareCallbacksConfig(
            root_dir=Path(config["root_dir"]),
            tensorbord_root_log_pair=tensorboard_root_log_dir,
            checkpoin_model_filepath=Path(config["updated_base_model_path"]),
        )


In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
# class PrepareCallback:
#     def __init__(self,config:PrepareCallbacksConfig):
#         self.config = config


#     @property
#     def _create_tb_callbacks_(self):
#         timetamp = time.strftime("%Y-%m-%d-%h-%M-%S")
#         tb_running_log_dir = os.path.join(
#             self.config.tensorbord_root_log_pai,
#             f"tb_logs_at_{timetamp}"
#         )
#         return tf.keras.callbacks.TensorBord(log_dir = tb_running_log_dir)
    

#     @property
#     def _create_ckpt_callbacks(self):
#       return tf.keras.callbacks.ModelCheckpoint(
#           filepath = self.config.checkpoin_model_filepath,
#           save_best_only = True
#       )
    
#     def get_to_ckpt_callbacks(self):
#         return[
#             self._create_tb_callbacks_,
#             self._create_ckpt_callbacks
#         ]





In [ ]:
from tensorflow import keras
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks_(self):
        timetamp = time.strftime("%Y-%m-%d-%H-%M-%S")  # Fixed timestamp typo
        tb_running_log_dir = os.path.join(
            self.config.tensorbord_root_log_pair,  # Correct attribute name
            f"tb_logs_at_{timetamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)  # Fixed TensorBoard typo

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.updated_base_model_path,  # Updated to use `.keras`
            save_best_only=True,
            save_weights_only=False,  # Adjust as needed
            save_format="keras"  # Explicitly specify the format
        )


    def get_to_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks_,
            self._create_ckpt_callbacks
        ]


In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_base_model_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_to_ckpt_callbacks()


except Exception as e:
    raise e

[2024-11-15 18:24:26,780: common: YAML file: d:\project\chicken-fecal-classification_project\config\config.yaml loaded successfully]
[2024-11-15 18:24:26,783: common: Content of d:\project\chicken-fecal-classification_project\config\config.yaml: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_injection', 'source_url': 'https://github.com/itisha249/chicken-fecal-classification_project/raw/refs/heads/main/archive%20(1).zip', 'local_data_file': 'artifacts/data_injection/data.zip', 'unzip_dir': 'artifacts/data_injection', 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}}]
[2024-11-15 18:24:26,788: common: YAML file: d:\project\chicken-fecal-classification_project\param.yaml loaded successfully]
[2024-11-15 18:24:26,790: common: Content of d:\project\chicken-fecal-classification_project\param.yaml:

ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=artifacts\prepare_base_model\base_model_updated.h5